In [ ]:
"""
Created on Fri Jan 12 11:21 2024

Wanted to look if can avoid the ice shelf divisions for plumes
@author: Clara Burgard
"""

In [ ]:
import multimelt.plume_functions as pf
import multimelt.useful_functions as uf

from tqdm.notebook import tqdm

from scipy.ndimage import gaussian_filter

import xarray as xr

In [ ]:
%matplotlib qt5

In [ ]:
map_lim = [-3000000,3000000]

#chunk_size = 700
chunk_size = False

READ IN DATA

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask ='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
outputpath_boxes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'
outputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'

file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_draft = file_draft.where(file_draft > 0, 0)
file_isf_conc = file_mask_orig_cut['isf_conc']
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice


xx = file_mask_orig_cut['x']
yy = file_mask_orig_cut['y']

xx = file_mask_orig_cut['x']
yy = file_mask_orig_cut['y']

dx = abs(xx[1] - xx[0])
dy = abs(yy[1] - yy[0])

file_isf = xr.open_dataset(outputpath_mask + 'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')



In [ ]:
ice_draft_pos = file_draft
# Be careful with ice shelf 178 and 195 - they have a negative ice draft
# I don't know how to fix it at the moment so I put it to nan
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 178, np.nan)
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 195, np.nan)

ice_draft_neg = -1*ice_draft_pos

In [ ]:
inputpath_plumes = '/bettik/burgardc/DATA/ERWIN_PAPER/interim/PLUMES/'

plume_charac = xr.open_dataset(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

In [ ]:
plume_charac['alpha'].sel(option='lazero') == 0)

In [ ]:
plume_charac['zGL'].sel(option='lazero').where((plume_charac['alpha'].sel(option='lazero') > 0) & (file_isf['ISF_mask'] > 1)).plot(vmax=0)

In [ ]:
((plume_charac['alpha'].sel(option='lazero') == 0) & (file_isf['ISF_mask'] > 1)).sum()

In [ ]:
def lazero_GL_alpha_kisf_newmethod2(kisf, ice_draft_neg_isf, GL_mask, isf_and_GL_mask, gl_mask_isl, dist_incl, weights8_0, weights16_0, mid_coord, sn_isf, first_crit, sn_isf_corr, first_crit_corr):
    """
    
    This function computes the plume departing grounding line depth and the local angle in a smoother manner than Lazeroms et al. 2018. 
    Remains heavily inspired from Lazeroms et al. 2018 (using the 16 directions).
    Includes an option to extend the grounding line to neighboring points in case the original grounding line is weirdly shallow.
    
    kisf : int
        ID of the ice shelf of interest
    ice_draft_neg : xr.DataArray
        Ice draft depth (Negative with depth!)
    GL_mask : xr.DataArray
        Mask of the Antarctic grounding lines
    isf_and_GL_mask : xr.DataArray  
        Mask of the isf and associated GL
    dist_incl : int
        Distance, in grid cells, to count within the grounding line
    weights8_0 : xr.Dataset
        Contains the weights (0,1) to look at the 8 neighbours of a point
    weights16_0 : xr.Dataset
        Contains the weights (0,1) to look in the 16 directions, starting at the point
    mid_coord : int
        Indication on how many times to propagate the grounding line
    sn_isf : xr.DataArray
        Slopes
    first_crit : xr.DataArray
       First criterion
    
    """
    
    # Enlarge GL mask to dist_incl rows (e.g. if your initial GL is shallow)
    GL_mask1_0 = (GL_mask == kisf)
    GL_2_mask = GL_mask1_0
    for n in range(dist_incl):
        GL_2 = pf.xr_nd_corr(GL_2_mask, weights8_0['weights'])
        GL_2_sum = GL_2.sum('direction').where(isf_and_GL_mask == kisf)
        GL_2_mask = (GL_2_sum > 0).astype(int)
        
    # Cut out the GL band in draft depth
    GL_depth_isf = -1*(ice_draft_neg_isf.where(GL_2_mask))
    


    # Initialise the field at grounding line
    GL_neighbors_new = GL_depth_isf
    sn_new = sn_isf.where(first_crit).mean('direction')
    sn_new = sn_new.where(sn_new > 0,0).where(GL_2_mask > 0)

    second_crit_all = GL_depth_isf * 0 + 1

    diff_masks = 1
    i = 0
    diff_stop = 0    


    #for ii in range(20):
    while diff_stop < 3:

        mask_old_domain = np.isnan(GL_neighbors_new)

        GL_neighbors = pf.xr_nd_corr(GL_neighbors_new, weights16_0['weights'])

        # cut out the newly formed data strip
        GL_neighbors_step = GL_neighbors.where(np.isnan(GL_neighbors_new))
        GL_neighbors_step = GL_neighbors_step.where(isf_and_GL_mask == kisf)

        # check if the propagated GL is deeper than point
        diff_base_GL = (-1*ice_draft_neg_isf - GL_neighbors_step)
        second_crit_n = diff_base_GL < 0 #<=

        # combine this criterion and the slope criterion
        all_crit =  first_crit & second_crit_n #

        if diff_masks != 0:

            # make a mean over all valid GL depths
            GL_mean = GL_neighbors_step.where(all_crit).mean('direction')
            GL_neighbors_new = GL_neighbors_new.where(GL_neighbors_new > 0,GL_mean)    

            # make a mean over all valid slopes
            sn_mean = sn_isf.where(all_crit).mean('direction')
            sn_new = sn_new.where(sn_new > 0,sn_mean)

            second_crit_all = second_crit_all.where(second_crit_all > 0,second_crit_n)   

            diff_stop = 0

        else:

            #print('Entering obstacle option')

            # insert corrected sn and first crit
            first_crit_corr2 = first_crit.where((all_crit.sum('direction') > 0), first_crit_corr)
            all_crit_corr = (first_crit_corr2 & second_crit_n).where(np.isfinite(GL_neighbors_step))
            sn_isf_corr2 = sn_isf.where((all_crit.sum('direction') > 0), sn_isf_corr).where(np.isfinite(GL_neighbors_step))

            # make a mean over all valid GL depths
            GL_mean = GL_neighbors_step.where(all_crit_corr).mean('direction')
            GL_neighbors_new = GL_neighbors_new.where(GL_neighbors_new > 0,GL_mean)    

            # make a mean over all valid slopes
            sn_mean = sn_isf_corr2.where(all_crit_corr).mean('direction')
            sn_new = sn_new.where(sn_new > 0,sn_mean)

            second_crit_all = second_crit_all.where(second_crit_all > 0,second_crit_n)   
            print('no island')

        if diff_stop == 2:
            print('island')
            
            
            #return gl_mask_isl, GL_mask, isf_and_GL_mask, GL_neighbors_new, second_crit_all, sn_new
            
            # cut out areas that are nan and potentially are near a grounding line of an island
            # modified this one below start_new_GL = (gl_mask_isl) &  ~(GL_mask == kisf) & (isf_and_GL_mask == kisf) #~(GL_neighbors_new > 0) 
            start_new_GL = (gl_mask_isl)  & ~(GL_mask == kisf) #& np.isnan(sn_new)

            GL_neighbors_new2 = -1*(ice_draft_neg_isf.where(start_new_GL))
            sn_new2 = sn_isf.where(first_crit).mean('direction')
            sn_new2 = sn_new2.where(sn_new2 > 0,0).where(start_new_GL > 0)
            second_crit_all2 = GL_neighbors_new2 * 0 + 1
            
            mask_old_domain2 = np.isnan(GL_neighbors_new2)
            
            diff_masks2 = 1

            for n in range(50):

                GL_neighbors = pf.xr_nd_corr(GL_neighbors_new2, weights16_0['weights'])

                # cut out the newly formed data strip
                GL_neighbors_step = GL_neighbors.where(np.isnan(GL_neighbors_new2))
                GL_neighbors_step = GL_neighbors_step.where(isf_and_GL_mask == kisf)

                # check if the propagated GL is deeper than point
                diff_base_GL = (-1*ice_draft_neg_isf - GL_neighbors_step)
                second_crit_n = diff_base_GL < 0 #<=

                # combine this criterion and the slope criterion
                all_crit =  first_crit & second_crit_n #
                
                if diff_masks2 != 0 :
                    # make a mean over all valid GL depths
                    GL_mean = GL_neighbors_step.where(all_crit).mean('direction')
                    GL_neighbors_new2 = GL_neighbors_new2.where(GL_neighbors_new2 > 0,GL_mean)    

                    # make a mean over all valid slopes
                    sn_mean = sn_isf.where(all_crit).mean('direction')
                    sn_new2 = sn_new2.where(sn_new2 > 0,sn_mean)

                    second_crit_all2 = second_crit_all2.where(second_crit_all2 > 0,second_crit_n)   
                    mask_new_domain2 = np.isnan(GL_neighbors_new2)
                    diff_masks2 = (mask_new_domain2.astype(int) - mask_old_domain2.astype(int)).sum().values
                    
                else:
                
                    # insert corrected sn and first crit
                    first_crit_corr2 = first_crit.where((all_crit.sum('direction') > 0), first_crit_corr)
                    all_crit_corr = (first_crit_corr2 & second_crit_n).where(np.isfinite(GL_neighbors_step))
                    sn_isf_corr2 = sn_isf.where((all_crit.sum('direction') > 0), sn_isf_corr).where(np.isfinite(GL_neighbors_step))

                    # make a mean over all valid GL depths
                    GL_mean = GL_neighbors_step.where(all_crit_corr).mean('direction')
                    GL_neighbors_new2 = GL_neighbors_new2.where(GL_neighbors_new2 > 0,GL_mean)    

                    # make a mean over all valid slopes
                    sn_mean = sn_isf_corr2.where(all_crit_corr).mean('direction')
                    sn_new2 = sn_new2.where(sn_new2 > 0,sn_mean)

                    second_crit_all2 = second_crit_all2.where(second_crit_all2 > 0,second_crit_n)
                    
                # fill nans with this new product
                GL_neighbors_new = GL_neighbors_new.combine_first(GL_neighbors_new2)
                sn_new = sn_new.combine_first(sn_new2)
            #return sn_new2, sn_mean

                ## fill nans with this new product
                #GL_neighbors_new = GL_neighbors_new.where(np.isfinite(GL_neighbors_new), GL_neighbors_new2)
            
        # check if we still have obstacles
        mask_new_domain = np.isnan(GL_neighbors_new)
        diff_masks = (mask_new_domain.astype(int) - mask_old_domain.astype(int)).sum().values

        # check if we have reached the maximum
        diff_mask_isf = np.isnan(GL_neighbors_new) & (isf_and_GL_mask == kisf)
        
        if diff_masks == 0:
            #print('mask did not change', diff_stop)
            diff_stop = diff_stop+1
                    
        i = i+1
        
        if i == 500:
            return  np.arctan(sn_new), -1*GL_neighbors_new
            print('reached 500 iterations')
            break
        
            
    return  np.arctan(sn_new), -1*GL_neighbors_new

In [ ]:
def first_criterion_lazero_general(kisf, plume_var_of_int, ice_draft_neg_isf, isf_and_GL_mask, ds_weights, dx, dy, dir_nb=16, grad_corr=0, extra_shift=2):

    """
    Define first criterion for the plume parameters using a smoother version of the 16 directions and permitting to use a different amount of directions
    
    This function computes the basal slope and identifies the first criterion, following the methodology in Lazeroms et al;, 2018.

    Parameters
    ----------
    kisf : int
        ID of the ice shelf of interest
    plume_var_of_int : xr.Dataset
        Dataset containing ``'ISF_mask'`` and ``'GL_mask'``
    ice_draft_neg_isf : xr.DataArray
        Ice draft depth for the given ice shelf in m. Negative downwards.
    isf_and_GL_mask : xr.DataArray
        Mask of the domain covered by the ice shelf and the grounding line (this extra mask is needed if the grounding line is defined on ground points)
    ds_weights : xr.Dataset
        Weights for the filter and information about the x- and y-shift in the 16 directions.
    dx : float
        Grid spacing in the x-direction
    dy : float
        Grid spacing in the y-direction
    dir_nb: int
        Amount of directions used. I tried with 8, 16, 24. Decided to stay with 16.
    grad_corr: int
        If we want to add some uncertainty in the slopes (adds grad_corr to the gradient) => makes it easier to have positive slopes when the differences are tiny.
    extra_shift: int
        Should be 2 if you do the smooth version, otherwise 1.
        
        
    Returns
    -------
    GL_depth : xr.DataArray
        Depth of the grounding line points (negative downwards).
    sn_isf : xr.DataArray
        Basal slope in all 16 directions
    first_crit : xr.DataArray
        Boolean where sn_sf > 0
    draft_depth : 
        Ice draft depth in m (negative downwards) extended through the 'direction' dimension.
    """
    
    # add dimension for directions to the ice_draft array
    other = xr.DataArray(np.zeros(dir_nb), coords=[('direction', np.arange(dir_nb))])
    ice_draft_neg_dirs, other2 = xr.broadcast(ice_draft_neg_isf, other)

    # draft depth only on the ice shelf
    draft_depth = ice_draft_neg_dirs.where(isf_and_GL_mask).where((plume_var_of_int['ISF_mask'] == kisf))

    # grounding line depth only where grounding line
    GL_depth = ice_draft_neg_dirs.where(isf_and_GL_mask).where(plume_var_of_int['GL_mask'] == kisf)
    GL_depth = GL_depth.where(GL_depth < 0, 0)
    
    # apply the correlation filter to compute gradients in the 16 directions (xr_nd_corr_sig does not work for whatever reason :( ))
    gradients = pf.xr_nd_corr(draft_depth, ds_weights['weights'])

    # compute the sn - basal slope - to be consistent with the origin of the plumes, we cut the basal slopes after ice shelves as well - but might need to think about what happens when several ice shelves are touching each other
    sn_isf = gradients / np.sqrt((ds_weights['shift_x'] * extra_shift * np.abs(dx)) ** 2 + (ds_weights['shift_y'] * extra_shift * np.abs(dy)) ** 2)
    # adding correction for criterion
    sn_isf_corr = (gradients +  grad_corr) / np.sqrt((ds_weights['shift_x'] * extra_shift * np.abs(dx)) ** 2 + (ds_weights['shift_y'] * extra_shift * np.abs(dy)) ** 2)
    # 1st criterion: sn > 0
    first_crit = sn_isf > 0
    first_crit_corr = sn_isf_corr > 0

    return sn_isf, sn_isf_corr, first_crit, first_crit_corr
  

In [ ]:
kisf = 42
extra_shift = 1
plume_var_of_int = file_isf
dir_nb = 16
grad_corr = 6
dist_incl= 2

In [ ]:
weights8 = pf.create_8_dir_weights()
weights16 = pf.create_16_dir_weights()

if dir_nb == 16:
    if extra_shift == 2:
        weights_across = pf.create_16_dir_weights_across()
    elif extra_shift == 1:
        weights_across = pf.create_16_dir_weights()
elif dir_nb == 8:
    if extra_shift == 2:
        weights_across = pf.create_8_dir_weights_across()
    elif extra_shift == 1:
        weights_across = pf.create_8_dir_weights()    


weights8_0 = weights8.where(weights8 < 0,0) * -1
weights8_0 = weights8_0.where(weights8_0 > 0,0)

weights16_0 = weights16.where(weights16 < 0,0) * -1
weights16_0 = weights16_0.where(weights16_0 > 0,0)


# prepare mask for whole domain (GL + ice shelf)
plume_var_of_int['GL_and_ISF_mask'] = plume_var_of_int['GL_mask'].combine_first(plume_var_of_int['ISF_mask'])
isf_and_GL_mask = plume_var_of_int['GL_and_ISF_mask'].where(
    (plume_var_of_int['ISF_mask'] == kisf) | (plume_var_of_int['GL_mask'] == kisf)).dropna(how='all',dim='x').dropna(how='all', dim='y')
ice_draft_neg_isf = ice_draft_neg.where(isf_and_GL_mask == kisf)

# first crit    
#draft_depth, ds_weights = first_criterion_lazero_general(kisf, plume_var_of_int, ice_draft_neg_isf, isf_and_GL_mask, weights_across, dx, dy, dir_nb=dir_nb, grad_corr=grad_corr, extra_shift=extra_shift) 
sn_isf, sn_isf_corr, first_crit, first_crit_corr = pf.first_criterion_lazero_general(kisf, plume_var_of_int, ice_draft_neg_isf, isf_and_GL_mask, weights_across, dx, dy, dir_nb=dir_nb, grad_corr=grad_corr, extra_shift=extra_shift) 


In [ ]:
plt.figure()
corr_mask_max.where(file_isf['ISF_mask'] == 11, drop=True).plot()

In [ ]:
weights4 = pf.create_4_dir_weights()
mask_0_1_2 = plume_var_of_int['ISF_mask'].where(plume_var_of_int['ISF_mask'] < 2,2)
corr_mask = pf.xr_nd_corr(mask_0_1_2, weights4['weights'])
corr_mask_max = np.abs(corr_mask).max('direction')
plume_var_of_int['GL_mask_with_isl'] = (corr_mask_max == 2)


#gl_mask_isl, GL_mask, isf_and_GL_mask, GL_neighbors_new, second_crit_all, sn_new 
alpha, zGL = pf.lazero_GL_alpha_kisf_newmethod2(kisf, 
                                             ice_draft_neg_isf, 
                                             plume_var_of_int['GL_mask'], 
                                             isf_and_GL_mask, 
                                             plume_var_of_int['GL_mask_with_isl'], 
                                             dist_incl, 
                                             weights8_0, 
                                             weights16_0, 
                                             200, 
                                             sn_isf, 
                                             first_crit, 
                                             sn_isf_corr, 
                                             first_crit_corr)


In [ ]:
weights4 = pf.create_4_dir_weights()
mask_0_1_2 = plume_var_of_int['ISF_mask'].where(plume_var_of_int['ISF_mask'] < 2,2)
corr_mask = pf.xr_nd_corr(mask_0_1_2, weights4['weights'])
corr_mask_max = np.abs(corr_mask).max('direction')
plume_var_of_int['GL_mask_with_isl'] = (corr_mask_max == 2)

alpha, zGL = pf.compute_zGL_alpha_lazero_newmethod(kisf, plume_var_of_int, ice_draft_neg, dx, dy, dir_nb, grad_corr, extra_shift, dist_incl)

In [ ]:
opt = 'new_lazero'
alpha, zGL = pf.compute_zGL_alpha_all(plume_var_of_int, opt, ice_draft_neg, grad_corr=grad_corr, dir_nb=dir_nb, extra_shift=extra_shift, dist_incl=dist_incl)

In [ ]:
plum_charac = pf.prepare_plume_charac([opt], ice_draft_pos, plume_var_of_int, grad_corr=grad_corr, dir_nb=dir_nb, extra_shift=extra_shift, dist_incl=dist_incl)

In [ ]:
#alpha_kisf = alpha.where(np.isfinite(alpha), 0)
#zGL_kisf = zGL.where(np.isfinite(zGL), ice_draft_neg_isf)

go_back_to_whole_grid_alpha = alpha.reindex_like(plume_var_of_int['ISF_mask'])
go_back_to_whole_grid_zgl = zGL.reindex_like(plume_var_of_int['ISF_mask'])   


In [ ]:
plum_charac['alpha'].plot()

In [ ]:
file_isf['ISF_mask'].plot(vmin=41,vmax=43)

In [ ]:
alpha.plot(vmax=0.005)

In [ ]:
((alpha == 0) & (file_isf['ISF_mask'] == kisf)).plot()

In [ ]:
alpha.where(np.isfinite(alpha), 0)

In [ ]:
plt.figure()
((alpha_kisf == 0) & (file_isf['ISF_mask'] == kisf)).plot()

In [ ]:
plt.figure()
file_isf['ISF_mask'].where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
for dd in range(4):
    plt.figure()
    np.abs(corr_mask).isel(direction=dd).where(file_isf['ISF_mask'] == 11, drop=True).plot()
    plt.title(str(dd))

In [ ]:
plt.figure()
np.abs(corr_mask).max('direction').where(file_isf['ISF_mask'] == 11, drop=True).plot()
plt.title(str(dd))

In [ ]:
((file_isf['ISF_mask'] == kisf) & (alpha > 0)).plot()

In [ ]:
plot()

In [ ]:
((gl_mask_isl) & ~(GL_mask == kisf)).plot()

In [ ]:
# cut out areas that are nan and potentially are near a grounding line of an island
start_new_GL = (gl_mask_isl) & ~(GL_mask == kisf)

GL_neighbors_new2 = -1*(ice_draft_neg_isf.where(start_new_GL))
sn_new2 = sn_isf.where(first_crit).mean('direction')
sn_new2 = sn_new2.where(sn_new2 > 0,0).where(start_new_GL > 0)
second_crit_all2 = GL_neighbors_new2 * 0 + 1

mask_old_domain2 = np.isnan(GL_neighbors_new2)

diff_masks2 = 1

In [ ]:
GL_neighbors = pf.xr_nd_corr(GL_neighbors_new2, weights16_0['weights'])

# cut out the newly formed data strip
GL_neighbors_step = GL_neighbors.where(np.isnan(GL_neighbors_new2))
GL_neighbors_step = GL_neighbors_step.where(isf_and_GL_mask == kisf)

# check if the propagated GL is deeper than point
diff_base_GL = (-1*ice_draft_neg_isf - GL_neighbors_step)
second_crit_n = diff_base_GL < 0 #<=

# combine this criterion and the slope criterion
all_crit =  first_crit & second_crit_n #

In [ ]:
for n in range(50):

    GL_neighbors = pf.xr_nd_corr(GL_neighbors_new2, weights16_0['weights'])

    # cut out the newly formed data strip
    GL_neighbors_step = GL_neighbors.where(np.isnan(GL_neighbors_new2))
    GL_neighbors_step = GL_neighbors_step.where(isf_and_GL_mask == kisf)

    # check if the propagated GL is deeper than point
    diff_base_GL = (-1*ice_draft_neg_isf - GL_neighbors_step)
    second_crit_n = diff_base_GL < 0 #<=

    # combine this criterion and the slope criterion
    all_crit =  first_crit & second_crit_n #

    if diff_masks2 != 0 :
        # make a mean over all valid GL depths
        GL_mean = GL_neighbors_step.where(all_crit).mean('direction')
        GL_neighbors_new2 = GL_neighbors_new2.where(GL_neighbors_new2 > 0,GL_mean)    

        # make a mean over all valid slopes
        sn_mean = sn_isf.where(all_crit).mean('direction')
        sn_new2 = sn_new2.where(sn_new2 > 0,sn_mean)

        second_crit_all2 = second_crit_all2.where(second_crit_all2 > 0,second_crit_n)   
        mask_new_domain2 = np.isnan(GL_neighbors_new2)
        diff_masks2 = (mask_new_domain2.astype(int) - mask_old_domain2.astype(int)).sum().values

    else:

        # insert corrected sn and first crit
        first_crit_corr2 = first_crit.where((all_crit.sum('direction') > 0), first_crit_corr)
        all_crit_corr = (first_crit_corr2 & second_crit_n).where(np.isfinite(GL_neighbors_step))
        sn_isf_corr2 = sn_isf.where((all_crit.sum('direction') > 0), sn_isf_corr).where(np.isfinite(GL_neighbors_step))

        # make a mean over all valid GL depths
        GL_mean = GL_neighbors_step.where(all_crit_corr).mean('direction')
        GL_neighbors_new2 = GL_neighbors_new2.where(GL_neighbors_new2 > 0,GL_mean)    

        # make a mean over all valid slopes
        sn_mean = sn_isf_corr2.where(all_crit_corr).mean('direction')
        sn_new2 = sn_new2.where(sn_new2 > 0,sn_mean)

        second_crit_all2 = second_crit_all2.where(second_crit_all2 > 0,second_crit_n)

In [ ]:
sn_new2.plot()

In [ ]:
alpha.plot()

In [ ]:
GL_neighbors_new_test = zGL.where(np.isfinite(zGL), alpha)

In [ ]:
zGL.combine_first(alpha).plot()

In [ ]:
GL_neighbors_new_test.plot()

In [ ]:
plt.figure()
alpha.plot()

In [ ]:
zGL.plot()

In [ ]:
zGL.plot()